In [34]:
#build confusion matrix

from collections import Counter, defaultdict
import pandas as pd
import pickle

#row in the confusion matrix
pickle1 = "/home/abc/ADM/pickle/Snippets.pkl"

#column in the confusion matrix
pickle2 = "/home/abc/ADM/pickle/Wikifier.pkl"


def createDictOfFromlistOfTuples(results):
    d = { file_id : { "l" : label, "p" : prob } for file_id, label , prob in results}
    return d

def merge2dicts(d1, d2):
    d1_keys = d1.keys()
    d2_keys = d2.keys()
        
    merged_d = dict()
    
    for key in d1_keys:
        if key in d2_keys:
            merged_d.update({ key : { "d1" : d1[key], "d2" : d2[key] } })
    return merged_d

#d1 and d2 are two dicts having the results of classifiers
def BuildConfusionMatrix(d1, d2, threshold):
    """
    unique_labels = sorted(set(results1[i][1] for i in range(len(results1) - 1)))
    d = dict()

    for label in unique_labels:
        d.update({label : Counter()})
    d.update({"No" : Counter()})
    """
    """
    for i in range(len(results1)):
        if results1[i][0] == results2[i][0]:
            if results1[i][2] < threshold and  results2[i][2] > threshold:
                d["No"].update([results2[i][2]])
            elif results1[i][2] > threshold and  results2[i][2] < threshold:
                d[results1[i][2]].update(["No"])
            elif results1[i][2] < threshold and  results2[i][2] < threshold:
                d["No"].update(["No"])
            else:
                d[results1[i]].update([results2[i]])
    return pd.DataFrame(d, index=unique_labels, columns=unique_labels)
    """
    m = defaultdict(lambda : 0)
    d2_keys = d2.keys()
    #print(d2_keys)
    for key in d2_keys:
        if isinstance(d2[key]["l"], str):
            m.update( { d2[key]["l"] : Counter() } )
            continue
        labels_2 = d2[key]["l"]
        for l in labels_2:
            if l not in m.keys():
                m.update( { l : Counter() } )
    #m.update( { "BT" : Counter() } )
    #print(m)
    bt_count = 0 #below threshold count
    for key in d2.keys():
        if key in d1.keys():
            if d2[key]["p"] > threshold and d1[key]["p"] > threshold:
                #print("Adding")
                #m[d2[key]["l"]].update( [str(d1[key]["l"])] )
                labels_2 = d2[key]["l"]
                labels_1 = d1[key]["l"]
                if isinstance(labels_2, str) and isinstance(labels_1, str):
                    m[d2[key]["l"]].update( [str(d1[key]["l"])] )
                else:
                    #both are Wikifier results
                    if not isinstance(labels_2, str) and not isinstance(labels_1, str):
                        #print("if")
                        for k_2 in labels_2:
                            for k_1 in labels_1:
                                m[k_2].update( [k_1])
                    #only first is wikifier results
                    elif not isinstance(labels_1, str):
                        #print("first elif")
                        for k_1 in d1[key]["l"]:
                            m[d2[key]["l"]].update( [k_1])
                    #only second is wikifier results
                    elif not isinstance(labels_2, str):
                        #print("second elif")
                        for k_2 in d2[key]["l"]:
                            m[k_2].update( [d1[key]["l"]])
            else:
                bt_count += 1
    print(bt_count)
    #print(m)
    return pd.DataFrame(m)
          

results1 = pickle.load(open(pickle1, 'rb'))
results2 = pickle.load(open(pickle2, 'rb'))

#print(results2)

if "Wikifier.pkl" not in pickle1:
    results_d1 = createDictOfFromlistOfTuples(results1)
else:
    results_d1 = results1
    
if "Wikifier.pkl" not in pickle2:
    results_d2 = createDictOfFromlistOfTuples(results2)
else:
    results_d2 = results2
"""
results_d1 = { "1" : {"l" : "arm", "p" : 0.3}, "2" : {"l" : "arch", "p" : 0.4}, "3" : {"l" : "soc", "p" : 0.7}, "4" : {"l" : "hist", "p" : 1} }
results_d2 = { "1" : {"l" : "psy", "p" : 0.4}, "2" : {"l" : "art", "p" : 0.5}, "3" : {"l" : "chem", "p" : 0.8}}
"""

cm_df = BuildConfusionMatrix(results_d1, results_d2, 0.2)
#print(cm_df)
writer = pd.ExcelWriter("Matrix.xlsx")
cm_df.to_excel(writer,"sheet1")
writer.save()
print("Done")

51
Done


In [21]:
d = dict()
d.update({1 : Counter()})
d[1].update(["abc"])
print(d)


{1: Counter({'abc': 1})}


In [3]:
d = {"one" : {"one" : 5, "two" : 2, "three" : 3},\
     "two" : {"one" : 1, "two" : 4, "three" : 2}, \
     "three" : {"one" : 0, "two" : 1, "three" : 7}}
df = pd.DataFrame(d)
print(df)

       one  three  two
one      5      0    1
three    3      7    2
two      2      1    4
